# Лаб.8 Ульченко 24ВВИм1

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "Qwen/Qwen2.5-7B-Instruct-1M"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

file_path = 'ENG_article.txt'
with open(file_path, 'r', encoding='utf-8') as file:
    data = file.read().replace('\n', ' ').strip()

print(f"Длина текста: {len(data)} символов")
print(f"Пример текста: {data[:200]}...")

def generate_response(messages, max_new_tokens=512):
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
    with torch.no_grad():
        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.1,
            top_p=0.9,
            do_sample=True,
            repetition_penalty=1.1
        )
    generated_ids_ = generated_ids[0][len(model_inputs.input_ids[0]):]
    response = tokenizer.decode(generated_ids_, skip_special_tokens=True)
    return response.strip()

system_prompt = {
    "role": "system",
    "content": "Ты полезный ассистент. Проанализируй предоставленный текст и ответь на вопрос точно, ссылаясь на содержание документа. Если информация отсутствует, укажи это явно. Ответь на русском языке, кратко и структурировано."
}

questions = [
    "В каком году была обозначена проблема взрывающихся градиентов?",
    "Кто в 1891 году разработал метод уничтожающей производной?",
    "Кто предложил цепное правило дифференцирования и в каком году?"
]

results = {}

for i, question in enumerate(questions, 1):
    messages = [
        system_prompt,
        {
            "role": "user",
            "content": f"Текст для анализа:\n{data}\n\nВопрос: {question}"
        }
    ]

    response = generate_response(messages)

    prompt_text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
    results[f"Вопрос {i}"] = {
        "prompt": prompt_text,
        "response": response
    }

    print(f"\n--- Вопрос {i}: {question} ---")
    print(f"Промпт: {prompt_text[:200]}...")
    print(f"Ответ: {response}")

with open('prompts_and_responses.txt', 'w', encoding='utf-8') as f:
    for key, value in results.items():
        f.write(f"{key}\n")
        f.write("Промпт:\n")
        f.write(value['prompt'] + "\n")
        f.write("Ответ:\n")
        f.write(value['response'] + "\n\n")

print("\nПромпты и ответы сохранены в 'prompts_and_responses.txt'")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

Длина текста: 8290 символов
Пример текста: Wikipedia https://en.m.wikipedia.org/wiki/Neural_network_(machine_learning) Neural network (machine learning)     This article is about the computational models used for artificial intelligence. For o...

--- Вопрос 1: В каком году была обозначена проблема взрывающихся градиентов? ---
Промпт: <|im_start|>system
Ты полезный ассистент. Проанализируй предоставленный текст и ответь на вопрос точно, ссылаясь на содержание документа. Если информация отсутствует, укажи это явно. Ответь на русском...
Ответ: Проблема взрывающихся градиентов была обозначена в 1991 году в дипломной работе Сеппа Хорецкого (Sepp Hochreiter).

--- Вопрос 2: Кто в 1891 году разработал метод уничтожающей производной? ---
Промпт: <|im_start|>system
Ты полезный ассистент. Проанализируй предоставленный текст и ответь на вопрос точно, ссылаясь на содержание документа. Если информация отсутствует, укажи это явно. Ответь на русском...
Ответ: Информация в предоставленном тексте не с